# SQL Notes - FGV Course
#### This notebook holds all the notes and exercises I took from the course.
#### It's a public knowledge source and a personal learning journal.

In [1]:
# Making sure that the notebook is running in the right directory

import os
os.chdir(r"C:\Users\grbar\OneDrive\Área de Trabalho\SQL_FGV\rep")
print(os.getcwd())


C:\Users\grbar\OneDrive\Área de Trabalho\SQL_FGV\rep


In [2]:
# Getting the data

%reload_ext sql
%sql sqlite:///data/curso_sql.db


#### **List of Tables**

In [3]:
%%sql
SELECT name 
FROM sqlite_master 
WHERE type='table';


 * sqlite:///data/curso_sql.db
Done.


name
fertility
gdp_pc
avg_income
co2_emissions_pc
life_expectancy
child_mortality
population
petrobras
women_years_at_school
men_years_at_school


## Working with **JOINS**

#### Consulting **GDP, Child Mortality, Average Income, Births Rate, Life Expectancy and Population** in Brazil in the years of 1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020.

##### My version using the table `country`, but we don't need it at this time, since another tables have 'country' column..

In [4]:
%%sql

SELECT  c.country as "pais", 
        gdp.ref_year AS "ano", 
        avgi.mean_usd as "pib", 
        cm.tot_deaths as "mort_inf", 
        gdp.gdp_pc as "renda_per_capita", 
        p.tot_pop as "populacao", 
        f.mean_babies as "nascimentos", 
        le.tot_years as "expec_vida" 
        
FROM country as c 

JOIN gdp_pc AS gdp ON c.country = gdp.country AND gdp.ref_year IN (1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2010,2020) 
JOIN avg_income AS avgi ON c.country = avgi.country AND avgi.ref_year = gdp.ref_year 
JOIN child_mortality AS cm ON c.country = cm.country AND cm.ref_year = gdp.ref_year 
JOIN population AS p ON c.country = p.country AND p.ref_year = gdp.ref_year 
JOIN fertility AS f ON c.country = f.country AND f.ref_year = gdp.ref_year 
JOIN life_expectancy AS le ON c.country = le.country AND le.ref_year = gdp.ref_year 

WHERE c.country = 'Brazil' ORDER BY ano ASC

 * sqlite:///data/curso_sql.db
Done.


pais,ano,pib,mort_inf,renda_per_capita,populacao,nascimentos,expec_vida
Brazil,1900,1.01,417,1250,18100000,5.88,31.1
Brazil,1910,1.14,410,1410,22400000,5.9,30.9
Brazil,1920,1.43,400,1770,27600000,5.9,30.8
Brazil,1930,1.56,391,1930,33800000,5.9,30.6
Brazil,1940,1.86,315,2300,41600000,5.9,34.8
Brazil,1950,2.51,212,3110,54000000,6.18,47.6
Brazil,1960,3.68,169,4550,73100000,6.21,52.4
Brazil,1970,5.12,133,6330,96400000,5.02,56.9
Brazil,1980,9.04,96.3,11200,122000000,4.07,62.7
Brazil,1990,9.6,63,10600,151000000,2.81,67.3


#### This version doesn't use the `country` table and columns, it just jumps to the information that we need. A way better :)

In [5]:
%%sql

SELECT 
    gdp.ref_year AS "ano",
    avgi.mean_usd AS "pib",
    cm.tot_deaths AS "mort_inf",
    gdp.gdp_pc AS "renda_per_capita",
    p.tot_pop AS "populacao",
    f.mean_babies AS "nascimentos",
    le.tot_years AS "expec_vida"
FROM gdp_pc gdp

JOIN avg_income avgi ON gdp.country = avgi.country AND gdp.ref_year = avgi.ref_year
JOIN child_mortality cm ON gdp.country = cm.country AND gdp.ref_year = cm.ref_year
JOIN population p ON gdp.country = p.country AND gdp.ref_year = p.ref_year
JOIN fertility f ON gdp.country = f.country AND gdp.ref_year = f.ref_year
JOIN life_expectancy le ON gdp.country = le.country AND gdp.ref_year = le.ref_year

WHERE gdp.country = 'Brazil' AND gdp.ref_year IN (1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2010,2020)
ORDER BY gdp.ref_year;


 * sqlite:///data/curso_sql.db
Done.


ano,pib,mort_inf,renda_per_capita,populacao,nascimentos,expec_vida
1900,1.01,417,1250,18100000,5.88,31.1
1910,1.14,410,1410,22400000,5.9,30.9
1920,1.43,400,1770,27600000,5.9,30.8
1930,1.56,391,1930,33800000,5.9,30.6
1940,1.86,315,2300,41600000,5.9,34.8
1950,2.51,212,3110,54000000,6.18,47.6
1960,3.68,169,4550,73100000,6.21,52.4
1970,5.12,133,6330,96400000,5.02,56.9
1980,9.04,96.3,11200,122000000,4.07,62.7
1990,9.6,63,10600,151000000,2.81,67.3


### LEFT JOINS 

#### Need to get child mortality rate and births rate on the six smallest countries in Europe (Andorra, Liechtenstein, Malta, Monaco, San Marino and Holy-See (Vatican)) for the year 2000


In [6]:
%%sql

SELECT cm.country, 
	   cm.tot_deaths, 
	   f.mean_babies
FROM child_mortality cm

LEFT JOIN fertility f ON cm.country = f.country AND cm.ref_year = f.ref_year

WHERE cm.country IN ( 'Andorra', 'Holy See', 'Liechtenstein', 'Malta', 'Monaco', 'San Marino') AND cm.ref_year= 2000;	

 * sqlite:///data/curso_sql.db
Done.


country,tot_deaths,mean_babies
Andorra,6.41,None
Holy See,37.4,None
Liechtenstein,11.5,None
Monaco,5.16,None
Malta,7.59,1.67
San Marino,6.07,None


#### Geting the same table as above, but with the population of each country for 2000

In [7]:
%%sql

SELECT cm.country, 
       cm.ref_year, 
       cm.tot_deaths, 
       f.mean_babies, 
       p.tot_pop
FROM child_mortality cm 

LEFT JOIN fertility f ON cm.country = f.country AND cm.ref_year = f.ref_year 
LEFT JOIN population p ON cm.country = p.country AND cm.ref_year = p.ref_year

WHERE cm.ref_year = 2000 AND cm.country IN ('Andorra', 'Holy See', 'Liechtenstein', 'Malta', 'Monaco', 'San Marino')

 * sqlite:///data/curso_sql.db
Done.


country,ref_year,tot_deaths,mean_babies,tot_pop
Andorra,2000,6.41,None,66100
Holy See,2000,37.4,None,779
Liechtenstein,2000,11.5,None,33000
Monaco,2000,5.16,None,32500
Malta,2000,7.59,1.67,399000
San Marino,2000,6.07,None,26800


#### Using **INNER JOIN** between `child_mortality` and `population` and then **LEFT JOIN** between `child_mortality` and `fertility`. This time the results are the same:

In [8]:
%%sql

SELECT cm.country, 
       cm.ref_year, 
       cm.tot_deaths, 
       f.mean_babies, 
       p.tot_pop
FROM child_mortality AS cm

INNER JOIN population p ON cm.country = p.country AND cm.ref_year = p.ref_year
LEFT JOIN fertility f ON cm.country = f.country AND cm.ref_year = f.ref_year

WHERE cm.ref_year = 2000 and cm.country IN ('Andorra', 'Holy See', 'Liechtenstein', 'Malta', 'Monaco', 'San Marino')

 * sqlite:///data/curso_sql.db
Done.


country,ref_year,tot_deaths,mean_babies,tot_pop
Andorra,2000,6.41,None,66100
Holy See,2000,37.4,None,779
Liechtenstein,2000,11.5,None,33000
Monaco,2000,5.16,None,32500
Malta,2000,7.59,1.67,399000
San Marino,2000,6.07,None,26800


##### Now I only change the order of the tables, using `fertility` first and then `child_mortality` with **RIGHT JOIN** being their correspondence

In [9]:
%%sql

SELECT 
    f.country, 
    cm.country, 
    cm.tot_deaths, 
    f.mean_babies
FROM fertility f
FULL JOIN child_mortality cm 
    ON cm.country = f.country 
    AND cm.ref_year = f.ref_year
WHERE 
    (f.country IN ('Aruba', 'Martinique', 'Puerto Rico', 'Dominica', 'Cuba')
     OR cm.country IN ('Aruba', 'Martinique', 'Puerto Rico', 'Dominica', 'Cuba'))
    AND (f.ref_year = 2000 OR cm.ref_year = 2000);


 * sqlite:///data/curso_sql.db
Done.


country,country_1,tot_deaths,mean_babies
Aruba,None,None,1.87
Cuba,Cuba,8.55,1.62
Martinique,None,None,1.9
Puerto Rico,None,None,1.91
None,Dominica,14.8,None


##### Notice how the filter `f.ref_year` = 2000 is applied inside the **ON** clause.
##### This ensures that the year condition affects only the **JOIN** matching, preserving rows from `child_mortality` even when no match exists in fertility.
##### If the same filter was placed in the **WHERE** clause, those rows would be dropped.

### **Full Join**

#### With the **FULL JOIN** method we can have all countries in the query, again using the `fertility` and `child_mortality` table we will get information of some Caribbean countries.

In [10]:
%%sql

SELECT f.country,
	   f.mean_babies,
	   cm.tot_deaths
FROM fertility f
FULL JOIN child_mortality cm ON f.country = cm.country AND f.ref_year = cm.ref_year
WHERE f.country IN ('Aruba','Cuba','Dominica','Martinique','Puerto Rico') AND f.ref_year = 2000

 * sqlite:///data/curso_sql.db
Done.


country,mean_babies,tot_deaths
Aruba,1.87,None
Cuba,1.62,8.55
Martinique,1.9,None
Puerto Rico,1.91,None


#### As a bonus point, we can use **COALESCE**, since the country Dominica is not present in `fertility`

In [ ]:
%%sql

SELECT 
    COALESCE(f.country, cm.country) AS country, 
    f.mean_babies, 
    cm.tot_deaths
FROM fertility f
FULL JOIN child_mortality cm 
    ON f.country = cm.country 
   AND f.ref_year = cm.ref_year
WHERE COALESCE(f.country, cm.country) IN ('Aruba', 'Cuba', 'Dominica', 'Martinique', 'Puerto Rico')
  AND (f.ref_year = 2000 OR cm.ref_year = 2000);


 * sqlite:///data/curso_sql.db
Done.


## Set Operations

### **UNION**

#### For the example, using **UNION** clause, we will get the BRICS countries (Brazil, Russia, India, China and South Africa) from `men_years_at_school` and `women_years_at_school` in the year of 2000.

In [ ]:
%%sql

SELECT m.country , m.ref_year, 'Male' AS gender, m.mean_years
FROM men_years_at_school m
WHERE m.ref_year = 2009 AND m.country IN ('Brazil','Russia','India','China','South Africa')
UNION
SELECT w.country, w.ref_year, 'Female' AS gender, w.mean_years
FROM women_years_at_school w
WHERE w.ref_year = 2009 AND w.country IN ('Brazil','Russia','India','China','South Africa')

 * sqlite:///data/curso_sql.db
Done.


country,ref_year,gender,mean_years
Brazil,2009,Female,7.2
Brazil,2009,Male,6.8
China,2009,Female,6.4
China,2009,Male,7.7
India,2009,Female,3.4
India,2009,Male,6.2
Russia,2009,Female,12.9
Russia,2009,Male,12.7
South Africa,2009,Female,8.5
South Africa,2009,Male,8.5


### **EXCEPT**

#### Finding which country is in `child_mortality` but not in `fertility`

In [ ]:
%%sql

SELECT cm.country FROM child_mortality cm
EXCEPT
SELECT f.country FROM fertility f
ORDER BY country ASC

 * sqlite:///data/curso_sql.db
Done.


country
Andorra
Dominica
Holy See
Liechtenstein
Marshall Islands
Monaco
Nauru
Palau
San Marino
St. Kitts and Nevis


#### Comparing the countries in `population` and `child_mortality`

In [ ]:
%%sql

SELECT p.country FROM population p
EXCEPT
SELECT cm.country FROM child_mortality cm

 * sqlite:///data/curso_sql.db
Done.


country


#### Both have the same countries

### **UNION ALL**

#### Using `co2_emissions_pc` and `men_years_at_school`, we will see if there was differences in the combination

In [ ]:
%%sql
SELECT *,'emissions' AS source_column
FROM co2_emissions_pc cep
WHERE cep.country = 'Brazil' AND cep.ref_year >= 2000
UNION ALL
SELECT *, 'men years at school' AS source_column
FROM men_years_at_school myas
WHERE myas.country = 'Brazil' AND myas.ref_year >= 2000
ORDER BY ref_year ASC

 * sqlite:///data/curso_sql.db
Done.


country,ref_year,co2_pc,source_column
Brazil,2000,1.97,emissions
Brazil,2000,5.7,men years at school
Brazil,2001,1.91,emissions
Brazil,2001,5.8,men years at school
Brazil,2002,1.82,emissions
Brazil,2002,6.0,men years at school
Brazil,2003,1.76,emissions
Brazil,2003,6.1,men years at school
Brazil,2004,1.81,emissions
Brazil,2004,6.2,men years at school


### **INTERSECT**

#### Getting the commom lines between `co2_emissions_pc` and `men_years_at_school`

In [ ]:
%%sql

SELECT cep.country, cep.ref_year 
FROM co2_emissions_pc cep 
WHERE cep.country = 'Brazil' AND cep.ref_year >= 2000

INTERSECT

SELECT myas.country, myas.ref_year 
FROM men_years_at_school myas 
WHERE myas.country = 'Brazil' AND myas.ref_year >= 2000

 * sqlite:///data/curso_sql.db
Done.


country,ref_year
Brazil,2000
Brazil,2001
Brazil,2002
Brazil,2003
Brazil,2004
Brazil,2005
Brazil,2006
Brazil,2007
Brazil,2008
Brazil,2009


## Summarization Functions


##### Using the `petrobras` stock market table, we will calculate the mean price and the range between the Close and Open prices. We will analyze the period between 2020-02-20 and 2020-03-20, as well as the dates on which the stock had a positive change.

In [ ]:
%%sql

SELECT Date, 
	   Open, 
	   High, 
	   Low, 
	   Close, 
	   "Adj Close", 
	   Volume,
	   (Close + Open)/2 AS mean,
	   (Close - Open) AS "range"
FROM petrobras

WHERE "range" > 0 AND Date BETWEEN "2020-02-20" AND "2020-03-20"


 * sqlite:///data/curso_sql.db
Done.


Date,Open,High,Low,Close,Adj Close,Volume,mean,range
2020-02-27,12.0,12.55,11.71,12.08,4.448638,41795900,12.04,0.08000000000000007
2020-02-28,11.56,12.11,11.51,12.1,4.456003,46881600,11.83,0.5399999999999991
2020-03-02,12.2,12.57,12.02,12.49,4.599626,29931200,12.344999999999999,0.2900000000000009
2020-03-19,4.4,5.12,4.1,5.06,1.863419,56339600,4.73,0.6599999999999993


#### Few days in that period that had upwards for the price 😅

#### Now its time to found the **African countries where woman spent at least 1 year or more than men in the school**. I have two approaches for that:

In [ ]:
%%sql

SELECT wyas.country,
	   wyas.ref_year,
	   wyas.mean_years AS "WYAS",
	   myas.mean_years AS "MYAS",
	   c.four_regions
FROM women_years_at_school wyas
LEFT JOIN country c ON c.country=wyas.country
LEFT JOIN men_years_at_school myas ON wyas.country = myas.country AND wyas.ref_year = myas.ref_year
WHERE c.four_regions = "africa" 
AND wyas.mean_years - myas.mean_years >= 1 
AND wyas.ref_year BETWEEN 2001 AND 2011

 * sqlite:///data/curso_sql.db
Done.


country,ref_year,WYAS,MYAS,four_regions
Lesotho,2001,6.9,5.0,africa
Lesotho,2002,7.1,5.1,africa
Lesotho,2003,7.3,5.2,africa
Lesotho,2004,7.5,5.3,africa
Lesotho,2005,7.7,5.4,africa
Lesotho,2006,7.9,5.5,africa
Lesotho,2007,8.1,5.7,africa
Lesotho,2008,8.2,5.8,africa
Lesotho,2009,8.4,5.9,africa
Seychelles,2001,10.1,8.6,africa


#### With this query we can analyze the number of years itself. 
#### Below we have only the **output of the countries**. Both ways are good, but they can be useful in differents scenarios.

In [ ]:
%%sql

SELECT DISTINCT m.country
FROM men_years_at_school m
JOIN women_years_at_school w ON m.country = w.country 
	AND m.ref_year = w.ref_year
JOIN country c ON m.country = c.country 
WHERE c.four_regions = 'africa'
	AND w.mean_years - m.mean_years >= 1
	AND w.ref_year BETWEEN 2001 AND 2010;

 * sqlite:///data/curso_sql.db
Done.


country
Lesotho
Seychelles


#### To use the arithmetic operator **%** (which is not percentage, but the modulus for division), we will retrieve the `child_mortality` table for **Brazil between 1910 and 2020, showing only the years at 10-year intervals (1910, 1920, 1930...)**.

In [ ]:
%%sql


SELECT country, 
       ref_year, 
       tot_deaths,
       100*tot_deaths/1000 AS perc_deaths
FROM child_mortality

WHERE ref_year BETWEEN 1910 and 2021 AND ref_year % 10 = 0 AND country = 'Brazil'

 * sqlite:///data/curso_sql.db
Done.


country,ref_year,tot_deaths,perc_deaths
Brazil,1910,410,41
Brazil,1920,400,40
Brazil,1930,391,39
Brazil,1940,315,31
Brazil,1950,212,21
Brazil,1960,169,16
Brazil,1970,133,13
Brazil,1980,96.3,9.63
Brazil,1990,63,6
Brazil,2000,34.8,3.4799999999999995


#### This operator is true power when showing samples..

#### Lets use **MIN(), MAX()** and **AVG()** to analyze the `life_expectancy` of the **Caribbean and Latin America countries in 1900**

In [ ]:
%%sql

SELECT c.country, 
	   le.ref_year, 
	   le.tot_years, 
	   AVG(le.tot_years) AS mean_tot_deaths, 
	   MAX(le.tot_years) AS max_tot_deaths,
	   MIN(le.tot_years) AS min_tot_deaths
	   
FROM country c
JOIN life_expectancy le ON le.country = c.country 
WHERE c.wb_regions = "Latin America & Caribbean" AND le.ref_year = 1900

 * sqlite:///data/curso_sql.db
Done.


country,ref_year,tot_years,mean_tot_deaths,max_tot_deaths,min_tot_deaths
Guatemala,1900,26.1,32.49032258064516,38.9,26.1


#### Using `avg_income` and `gdp_pc` with **AVG()**, we will calculate the **average GDP per capita and average income per capita for all countries in 2019, rounded to two decimal places.**

In [ ]:
%%sql

SELECT ai.ref_year,
       ROUND(AVG(ai.mean_usd),2) as avg_mean_usd,
       ROUND(AVG(gp.gdp_pc),2) as avg_gdp_pc
FROM avg_income ai
JOIN gdp_pc gp ON ai.country=gp.country AND ai.ref_year=gp.ref_year 
WHERE ai.ref_year = 2019

#### Now we will **calculate the average difference in years of schooling between men and women in the year 2000**, using the tables `men_years_at_school` and `women_years_at_school`.

In [7]:
%%sql

SELECT ROUND(AVG(myas.mean_years-wyas.mean_years),2) AS men_mean_years,
	   myas.ref_year
FROM men_years_at_school myas 
JOIN women_years_at_school wyas ON myas.country=wyas.country AND myas.ref_year=wyas.ref_year 
WHERE myas.ref_year = 2000

 * sqlite:///data/curso_sql.db
Done.


men_mean_years,ref_year
1.09,2000


### AGGREGATION FUNCTIONS (GROUP BY and ORDER BY)

#### These functions work together with aggregation.  
#### You can use **MAX**, **MIN**, and **AVG** alongside the **GROUP BY** clause to summarize data.

In [5]:
%%sql

SELECT 
    c.wb3income,
    MAX(ai.mean_usd) AS max_income,
    MIN(ai.mean_usd) AS min_income,
    AVG(ai.mean_usd) AS avg_income
FROM avg_income ai
JOIN country c ON ai.country = c.country
WHERE ai.ref_year = 2020
GROUP BY c.wb3income
ORDER BY avg_income DESC;

 * sqlite:///data/curso_sql.db
Done.


wb3income,max_income,min_income,avg_income
High income,112.0,13.9,50.48070175438597
Middle income,32.2,2.24,11.704380952380953
Low income,7.28,1.53,3.763225806451613
